# EX Data Analysis

In [1]:
import pandas as pd
import math
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from factor_analyzer import (
    ConfirmatoryFactorAnalyzer, ModelSpecificationParser)
from scipy.stats import sem
from scipy.stats import f_oneway
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from pyprocessmacro import Process


In [2]:
# load data
data = pd.read_csv("Summary.csv")
# print(data.describe())
print(data.columns)

Index(['Group', 'T-P', 'T-T', 'UserID', 'No', 'IP', 'Started', 'Ended', 'Pid',
       'PRH1_1', 'PRH2_1', 'PRH3_1', 'CPFM1_1', 'CPF1_1', 'CPF2_1', 'CPFM2_1',
       'CPF3_1', 'PRC1_1', 'PRC2_1', 'PRC3_1', 'PRC4_1', 'PRCM1_1', 'PRE1_1',
       'PRE2_1', 'PR1_1', 'PR2_1', 'F1', 'F2', 'F3', 'F4', 'F5', 'AC1', 'F6',
       'F7', 'PRH1_2', 'PRH2_2', 'PRH3_2', 'CPFM1_2', 'CPF1_2', 'CPF2_2',
       'CPFM2_2', 'CPF3_2', 'PRC1_2', 'PRC2_2', 'PRC3_2', 'PRC4_2', 'PRCM1_2',
       'PRE1_2', 'PRE2_2', 'PR1_2', 'PR2_2', 'AC2', 'PRH1_3', 'PRH2_3',
       'PRH3_3', 'CPFM1_3', 'CPF1_3', 'CPF2_3', 'CPFM2_3', 'CPF3_3', 'PRC1_3',
       'PRC2_3', 'PRC3_3', 'PRC4_3', 'PRCM1_3', 'PRE1_3', 'PRE2_3', 'PR1_3',
       'PR2_3', 'PV_1', 'TV_1', 'PV_2', 'TV_2', 'PV_3', 'TV_3', 'PrePV_1',
       'PreTV_1', 'PrePV_2', 'PreTV_2', 'PrePV_3', 'PreTV_3', 'Gender', 'Age',
       'Education', 'Income', 'Race', 'Ideology', 'Satiety'],
      dtype='object')


## Balance Check

In [3]:
data.groupby("Group")["Gender"].value_counts()
data.groupby("Group")["Age"].value_counts()
data.groupby("Group")["Education"].value_counts()
data.groupby("Group")["Income"].value_counts()
data.groupby("Group")["Race"].value_counts()
data.groupby("Group")["Ideology"].value_counts()
data.groupby("Group")["Satiety"].value_counts()
cross_tab = {attri: pd.crosstab(data["Group"], data[attri])
             for attri in ["Gender", "Age", "Income", "Education", "Race", "Ideology", "Satiety"]}
balance_tab = pd.concat(cross_tab.values(), axis=1)

chisq_res = {attri: chi2_contingency(cross_tab[attri])
for attri in ["Gender", "Age", "Income", "Education", "Race", "Ideology", "Satiety"]}
print(chisq_res)

{'Gender': (51.794708807433835, 0.5599085531118048, 54, array([[20.06167401, 25.33039648,  0.40528634,  0.20264317],
       [21.37004405, 26.98237885,  0.43171806,  0.21585903],
       [21.8061674 , 27.53303965,  0.44052863,  0.22026432],
       [21.8061674 , 27.53303965,  0.44052863,  0.22026432],
       [21.8061674 , 27.53303965,  0.44052863,  0.22026432],
       [17.88105727, 22.57709251,  0.36123348,  0.18061674],
       [26.16740088, 33.03964758,  0.52863436,  0.26431718],
       [22.24229075, 28.08370044,  0.44933921,  0.2246696 ],
       [17.88105727, 22.57709251,  0.36123348,  0.18061674],
       [20.9339207 , 26.43171806,  0.42290749,  0.21145374],
       [19.18942731, 24.22907489,  0.3876652 ,  0.1938326 ],
       [24.42290749, 30.83700441,  0.49339207,  0.24669604],
       [26.16740088, 33.03964758,  0.52863436,  0.26431718],
       [22.6784141 , 28.63436123,  0.45814978,  0.22907489],
       [17.00881057, 21.47577093,  0.34361233,  0.17180617],
       [21.8061674 , 27.53303

- Gender: (50.817407566107775, 0.5979139739901153, 54)
- Age: (52.28537878421162, 0.5407771946256807, 54)
- Income: (63.004015406568335, 0.18785751858850525, 54)
- Education: (90.18966246732725, 0.47455092760799406, 90)
- Race: (147.96552059172564, 0.3932851622863128, 144)
- Ideology: (121.500764476331, 0.5966682833209155, 126)
- Satiety: (139.2766050851772, **0.022966236027190935**, 108)

## Reliability

In [4]:
def C_alpha(var_frame):
    cors = var_frame.corr()
    N = cors.shape[0]
    rs = np.array([])
    for i, col in enumerate(cors.columns):
        sum_ = cors[col][i+1:].values
        rs = np.append(sum_, rs)
    mean_r = np.mean(rs)
    cronbach_alpha = (N * mean_r) / (1 + (N - 1) * mean_r)
    return cronbach_alpha

### Review 1

In [5]:
data["CPF4_1"] = 7 - data["CPFM1_1"]
data["CPF5_1"] = 7 - data["CPFM2_1"]
print(C_alpha(data[["PRH1_1", "PRH2_1", "PRH3_1"]]))
print(C_alpha(data[["CPF1_1", "CPF2_1","CPF3_1"]]))
print(C_alpha(data[["CPF1_1", "CPF2_1","CPF3_1","CPF4_1","CPF5_1"]]))
print(C_alpha(data[["PRC1_1", "PRC2_1","PRC3_1","PRC4_1"]]))
print(C_alpha(data[["PRE1_1", "PRE2_1"]]))
print(C_alpha(data[["PR1_1", "PR2_1"]]))

0.9094922936848137
0.9121477612716057
0.9227523248826658
0.8588987787710568
0.953277839632788
0.9267808874586236


### Review 2

In [6]:
data["CPF4_2"] = 7 - data["CPFM1_2"]
data["CPF5_2"] = 7 - data["CPFM2_2"]
print(C_alpha(data[["PRH1_2", "PRH2_2", "PRH3_2"]]))
print(C_alpha(data[["CPF1_2", "CPF2_2","CPF3_2"]]))
print(C_alpha(data[["CPF1_2", "CPF2_2","CPF3_2","CPF4_2","CPF5_2"]]))
print(C_alpha(data[["PRC1_2", "PRC2_2","PRC3_2","PRC4_2"]]))
print(C_alpha(data[["PRE1_2", "PRE2_2"]]))
print(C_alpha(data[["PR1_2", "PR2_2"]]))

0.9017871146533515
0.9091177957337073
0.9239496687011874
0.8500660577806488
0.9504329202567414
0.9303448565910363


### Review 3

In [7]:
data["CPF4_3"] = 7 - data["CPFM1_3"]
data["CPF5_3"] = 7 - data["CPFM2_3"]
print(C_alpha(data[["PRH1_3", "PRH2_3", "PRH3_3"]]))
print(C_alpha(data[["CPF1_3", "CPF2_3","CPF3_3"]]))
print(C_alpha(data[["CPF1_3", "CPF2_3","CPF3_3","CPF4_3","CPF5_3"]]))
print(C_alpha(data[["PRC1_3", "PRC2_3","PRC3_3","PRC4_3"]]))
print(C_alpha(data[["PRE1_3", "PRE2_3"]]))
print(C_alpha(data[["PR1_3", "PR2_3"]]))

0.891896186042029
0.8927425746240123
0.9135408253966733
0.79658908905611
0.9454436732526684
0.9077287109560324


## Validity
### Review 1
#### With inverse items

In [8]:
temp = data[["PRH1_1", "PRH2_1", "PRH3_1", 
"CPF1_1", "CPF2_1","CPF3_1","CPF4_1","CPF5_1",
"PRC1_1", "PRC2_1","PRC3_1","PRC4_1",
"PRE1_1", "PRE2_1",
"PR1_1", "PR2_1"]]
model_dict = {"F1": ["PRH1_1", "PRH2_1", "PRH3_1"],
"F2": ["CPF1_1", "CPF2_1","CPF3_1","CPF4_1","CPF5_1"],
"F3": ["PRC1_1", "PRC2_1","PRC3_1","PRC4_1"],
"F4": ["PRE1_1", "PRE2_1"],
"F5": ["PR1_1", "PR2_1"],
}
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(
    temp, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)
cfa.fit(temp.values)
loadings = cfa.loadings_
loadings_se, error_vars_se = cfa.get_standard_errors()
print(loadings_se)
print(error_vars_se)

[[0.03788106 0.         0.         0.         0.        ]
 [0.0374847  0.         0.         0.         0.        ]
 [0.03963094 0.         0.         0.         0.        ]
 [0.         0.03653547 0.         0.         0.        ]
 [0.         0.03679159 0.         0.         0.        ]
 [0.         0.04058037 0.         0.         0.        ]
 [0.         0.0414884  0.         0.         0.        ]
 [0.         0.039035   0.         0.         0.        ]
 [0.         0.         0.03777792 0.         0.        ]
 [0.         0.         0.04039156 0.         0.        ]
 [0.         0.         0.0401621  0.         0.        ]
 [0.         0.         0.04519059 0.         0.        ]
 [0.         0.         0.         0.04075717 0.        ]
 [0.         0.         0.         0.04137851 0.        ]
 [0.         0.         0.         0.         0.0423174 ]
 [0.         0.         0.         0.         0.04284178]]
[0.03798752 0.03725738 0.04429894 0.03317954 0.03348396 0.04444629
 0.0

c:\Users\wxy\anaconda3\envs\myenv\lib\site-packages\factor_analyzer\confirmatory_factor_analyzer.py:753: UserWarning: The optimization routine failed to converge: ABNORMAL_TERMINATION_IN_LNSRCH
  warnings.warn(


In [9]:
error_vars_se = np.sqrt(cfa.error_vars_.reshape(16,))
print(cfa.factor_varcovs_)

[[1.         0.35281294 0.54674435 0.34140372 0.35104944]
 [0.35281294 1.         0.41854684 0.12703559 0.37898518]
 [0.54674435 0.41854684 1.         0.30901493 0.40752946]
 [0.34140372 0.12703559 0.30901493 1.         0.56914143]
 [0.35104944 0.37898518 0.40752946 0.56914143 1.        ]]


In [10]:
loads = loadings.reshape(80,)[[0,5,10,16,21,26,31,36,42,47,52,57,63,68,74,79]]
# print(loads)
print(sum(loads[:3]**2)/(sum(loads[:3]**2)+ sum(error_vars_se[:3]**2)))
print(sum(loads[3:8]**2)/(sum(loads[3:8]**2)+ sum(error_vars_se[3:8]**2)))
print(sum(loads[8:12]**2)/(sum(loads[8:12]**2)+ sum(error_vars_se[8:12]**2)))
print(sum(loads[12:14]**2)/(sum(loads[12:14]**2)+ sum(error_vars_se[12:14]**2)))
print(sum(loads[14:16]**2)/(sum(loads[14:16]**2)+ sum(error_vars_se[14:16]**2)))

0.6892897459351771
0.6238902573800602
0.5740247938073572
0.8099473911950155
0.7592110547288565


#### Without inverse items

In [11]:
temp = data[["PRH1_1", "PRH2_1", "PRH3_1", 
"CPF1_1", "CPF2_1","CPF3_1",
"PRC1_1", "PRC2_1","PRC3_1","PRC4_1",
"PRE1_1", "PRE2_1",
"PR1_1", "PR2_1"]]
model_dict = {"F1": ["PRH1_1", "PRH2_1", "PRH3_1"],
"F2": ["CPF1_1", "CPF2_1","CPF3_1",],
"F3": ["PRC1_1", "PRC2_1","PRC3_1","PRC4_1"],
"F4": ["PRE1_1", "PRE2_1"],
"F5": ["PR1_1", "PR2_1"],
}
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(
    temp, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)
cfa.fit(temp.values)
loadings = cfa.loadings_
loadings_se, error_vars_se = cfa.get_standard_errors()
error_vars_se = np.sqrt(cfa.error_vars_.reshape(14,))
print(cfa.factor_varcovs_)
loads = loadings.reshape(70,)[[0,5,10,16,21,26,32,37,42,47,53,58,64,69]]
# print(loads)
print(sum(loads[:3]**2)/(sum(loads[:3]**2)+ sum(error_vars_se[:3]**2)))
print(sum(loads[3:6]**2)/(sum(loads[3:6]**2)+ sum(error_vars_se[3:6]**2)))
print(sum(loads[6:10]**2)/(sum(loads[6:10]**2)+ sum(error_vars_se[6:10]**2)))
print(sum(loads[10:12]**2)/(sum(loads[10:12]**2)+ sum(error_vars_se[10:12]**2)))
print(sum(loads[12:14]**2)/(sum(loads[12:14]**2)+ sum(error_vars_se[12:14]**2)))

[[1.         0.37030537 0.56830425 0.35264104 0.36680655]
 [0.37030537 1.         0.44857457 0.1373437  0.35714247]
 [0.56830425 0.44857457 1.         0.31942085 0.42807066]
 [0.35264104 0.1373437  0.31942085 1.         0.58785272]
 [0.36680655 0.35714247 0.42807066 0.58785272 1.        ]]
0.6904715681721245
0.6936653493806221
0.5704701774684252
0.814557419155247
0.7621347199132245


c:\Users\wxy\anaconda3\envs\myenv\lib\site-packages\factor_analyzer\confirmatory_factor_analyzer.py:753: UserWarning: The optimization routine failed to converge: ABNORMAL_TERMINATION_IN_LNSRCH
  warnings.warn(


### Review 2
#### With inverse items

In [12]:
temp = data[["PRH1_2", "PRH2_2", "PRH3_2", 
"CPF1_2", "CPF2_2","CPF3_2","CPF4_2","CPF5_2",
"PRC1_2", "PRC2_2","PRC3_2","PRC4_2",
"PRE1_2", "PRE2_2",
"PR1_2", "PR2_2"]]
model_dict = {"F1": ["PRH1_2", "PRH2_2", "PRH3_2"],
"F2": ["CPF1_2", "CPF2_2","CPF3_2","CPF4_2","CPF5_2"],
"F3": ["PRC1_2", "PRC2_2","PRC3_2","PRC4_2"],
"F4": ["PRE1_2", "PRE2_2"],
"F5": ["PR1_2", "PR2_2"],
}
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(
    temp, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)
cfa.fit(temp.values)
loadings = cfa.loadings_
loadings_se, error_vars_se = cfa.get_standard_errors()
error_vars_se = np.sqrt(cfa.error_vars_.reshape(16,))
print(cfa.factor_varcovs_)
loads = loadings.reshape(80,)[[0,5,10,16,21,26,31,36,42,47,52,57,63,68,74,79]]
# print(loads)
print(sum(loads[:3]**2)/(sum(loads[:3]**2)+ sum(error_vars_se[:3]**2)))
print(sum(loads[3:8]**2)/(sum(loads[3:8]**2)+ sum(error_vars_se[3:8]**2)))
print(sum(loads[8:12]**2)/(sum(loads[8:12]**2)+ sum(error_vars_se[8:12]**2)))
print(sum(loads[12:14]**2)/(sum(loads[12:14]**2)+ sum(error_vars_se[12:14]**2)))
print(sum(loads[14:16]**2)/(sum(loads[14:16]**2)+ sum(error_vars_se[14:16]**2)))

[[1.         0.37437088 0.49136019 0.32907304 0.37572412]
 [0.37437088 1.         0.42025426 0.27345319 0.36115347]
 [0.49136019 0.42025426 1.         0.25666793 0.40688664]
 [0.32907304 0.27345319 0.25666793 1.         0.71794502]
 [0.37572412 0.36115347 0.40688664 0.71794502 1.        ]]
0.6778811697575476
0.61804808736426
0.5679909446043278
0.8103350783134341
0.7704196610256366


c:\Users\wxy\anaconda3\envs\myenv\lib\site-packages\factor_analyzer\confirmatory_factor_analyzer.py:753: UserWarning: The optimization routine failed to converge: ABNORMAL_TERMINATION_IN_LNSRCH
  warnings.warn(


#### Without inverse items

In [13]:
temp = data[["PRH1_2", "PRH2_2", "PRH3_2", 
"CPF1_2", "CPF2_2","CPF3_2",
"PRC1_2", "PRC2_2","PRC3_2","PRC4_2",
"PRE1_2", "PRE2_2",
"PR1_2", "PR2_2"]]
model_dict = {"F1": ["PRH1_2", "PRH2_2", "PRH3_2"],
"F2": ["CPF1_2", "CPF2_2","CPF3_2",],
"F3": ["PRC1_2", "PRC2_2","PRC3_2","PRC4_2"],
"F4": ["PRE1_2", "PRE2_2"],
"F5": ["PR1_2", "PR2_2"],
}
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(
    temp, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)
cfa.fit(temp.values)
loadings = cfa.loadings_
loadings_se, error_vars_se = cfa.get_standard_errors()
error_vars_se = np.sqrt(cfa.error_vars_.reshape(14,))
print(cfa.factor_varcovs_)
loads = loadings.reshape(70,)[[0,5,10,16,21,26,32,37,42,47,53,58,64,69]]
# print(loads)
print(sum(loads[:3]**2)/(sum(loads[:3]**2)+ sum(error_vars_se[:3]**2)))
print(sum(loads[3:6]**2)/(sum(loads[3:6]**2)+ sum(error_vars_se[3:6]**2)))
print(sum(loads[6:10]**2)/(sum(loads[6:10]**2)+ sum(error_vars_se[6:10]**2)))
print(sum(loads[10:12]**2)/(sum(loads[10:12]**2)+ sum(error_vars_se[10:12]**2)))
print(sum(loads[12:14]**2)/(sum(loads[12:14]**2)+ sum(error_vars_se[12:14]**2)))

[[1.         0.61395396 0.76417567 0.54789192 0.63128136]
 [0.61395396 1.         0.69493835 0.48786514 0.57392346]
 [0.76417567 0.69493835 1.         0.55251512 0.68532066]
 [0.54789192 0.48786514 0.55251512 1.         0.81381909]
 [0.63128136 0.57392346 0.68532066 0.81381909 1.        ]]
0.7661093554661047
0.7682140317372926
0.596312570550698
0.908985136539126
0.8697801745884088


### Review 3
#### With inverse items

In [14]:
temp = data[["PRH1_3", "PRH2_3", "PRH3_3", 
"CPF1_3", "CPF2_3","CPF3_3","CPF4_3","CPF5_3",
"PRC1_3", "PRC2_3","PRC3_3","PRC4_3",
"PRE1_3", "PRE2_3",
"PR1_3", "PR2_3"]]
model_dict = {"F1": ["PRH1_3", "PRH2_3", "PRH3_3"],
"F2": ["CPF1_3", "CPF2_3","CPF3_3","CPF4_3","CPF5_3"],
"F3": ["PRC1_3", "PRC2_3","PRC3_3","PRC4_3"],
"F4": ["PRE1_3", "PRE2_3"],
"F5": ["PR1_3", "PR2_3"],
}
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(
    temp, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)
cfa.fit(temp.values)
loadings = cfa.loadings_
loadings_se, error_vars_se = cfa.get_standard_errors()
error_vars_se = np.sqrt(cfa.error_vars_.reshape(16,))
print(cfa.factor_varcovs_)
loads = loadings.reshape(80,)[[0,5,10,16,21,26,31,36,42,47,52,57,63,68,74,79]]
# print(loads)
print(sum(loads[:3]**2)/(sum(loads[:3]**2)+ sum(error_vars_se[:3]**2)))
print(sum(loads[3:8]**2)/(sum(loads[3:8]**2)+ sum(error_vars_se[3:8]**2)))
print(sum(loads[8:12]**2)/(sum(loads[8:12]**2)+ sum(error_vars_se[8:12]**2)))
print(sum(loads[12:14]**2)/(sum(loads[12:14]**2)+ sum(error_vars_se[12:14]**2)))
print(sum(loads[14:16]**2)/(sum(loads[14:16]**2)+ sum(error_vars_se[14:16]**2)))

[[1.         0.40502014 0.4393153  0.32875957 0.2523393 ]
 [0.40502014 1.         0.35961138 0.24565603 0.32691379]
 [0.4393153  0.35961138 1.         0.25791803 0.29989612]
 [0.32875957 0.24565603 0.25791803 1.         0.65058493]
 [0.2523393  0.32691379 0.29989612 0.65058493 1.        ]]
0.6735356661442742
0.6113551329303143
0.5353705641251695
0.813970362381772
0.7477418319263505


c:\Users\wxy\anaconda3\envs\myenv\lib\site-packages\factor_analyzer\confirmatory_factor_analyzer.py:753: UserWarning: The optimization routine failed to converge: ABNORMAL_TERMINATION_IN_LNSRCH
  warnings.warn(


#### Without inverse items

In [15]:
temp = data[["PRH1_3", "PRH2_3", "PRH3_3", 
"CPF1_3", "CPF2_3","CPF3_3",
"PRC1_3", "PRC2_3","PRC3_3","PRC4_3",
"PRE1_3", "PRE2_3",
"PR1_3", "PR2_3"]]
model_dict = {"F1": ["PRH1_3", "PRH2_3", "PRH3_3"],
"F2": ["CPF1_3", "CPF2_3","CPF3_3",],
"F3": ["PRC1_3", "PRC2_3","PRC3_3","PRC4_3"],
"F4": ["PRE1_3", "PRE2_3"],
"F5": ["PR1_3", "PR2_3"],
}
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(
    temp, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)
cfa.fit(temp.values)
loadings = cfa.loadings_
loadings_se, error_vars_se = cfa.get_standard_errors()
error_vars_se = np.sqrt(cfa.error_vars_.reshape(14,))
print(cfa.factor_varcovs_)
loads = loadings.reshape(70,)[[0,5,10,16,21,26,32,37,42,47,53,58,64,69]]
# print(loads)
print(sum(loads[:3]**2)/(sum(loads[:3]**2)+ sum(error_vars_se[:3]**2)))
print(sum(loads[3:6]**2)/(sum(loads[3:6]**2)+ sum(error_vars_se[3:6]**2)))
print(sum(loads[6:10]**2)/(sum(loads[6:10]**2)+ sum(error_vars_se[6:10]**2)))
print(sum(loads[10:12]**2)/(sum(loads[10:12]**2)+ sum(error_vars_se[10:12]**2)))
print(sum(loads[12:14]**2)/(sum(loads[12:14]**2)+ sum(error_vars_se[12:14]**2)))

[[1.         0.62524597 0.73248192 0.52622446 0.50393407]
 [0.62524597 1.         0.64108197 0.45705074 0.5044184 ]
 [0.73248192 0.64108197 1.         0.51219328 0.56692016]
 [0.52622446 0.45705074 0.51219328 1.         0.77050168]
 [0.50393407 0.5044184  0.56692016 0.77050168 1.        ]]
0.7393941850001271
0.7380280404428456
0.5009110715965704
0.8974838539528909
0.8311185923990638


## Manipulation Check
### Review 1
#### Picture

In [16]:
pic_data = data[data["T-P"] != "N"]
txt_data = data[data["T-T"] != "N"]
tv,pv = stats.ttest_ind(pic_data[pic_data["T-P"] == "B"]["PV_1"], pic_data[pic_data["T-P"] == "H"]["PV_1"])
print(str(pv/2))
print(pic_data[pic_data["T-P"] == "B"]["PV_1"].mean())
print(pic_data[pic_data["T-P"] == "H"]["PV_1"].mean())
print(pic_data[pic_data["T-P"] == "VH"]["PV_1"].mean())

6.330204361127538e-11
4.789029535864979
5.54978354978355
5.96137339055794


#### Text

In [17]:
tv,pv = stats.ttest_ind(txt_data[txt_data["T-T"] == "H"]["TV_1"], txt_data[txt_data["T-T"] == "VH"]["TV_1"])
print(str(pv/2))
print(pic_data[pic_data["T-T"] == "L"]["TV_1"].mean())
print(pic_data[pic_data["T-T"] == "B"]["TV_1"].mean())
print(pic_data[pic_data["T-T"] == "H"]["TV_1"].mean())
print(pic_data[pic_data["T-T"] == "VH"]["TV_1"].mean())

0.004988238402119758
3.0206896551724136
4.973509933774834
5.473684210526316
5.735099337748345


### Review 2
#### Picture

In [18]:
tv,pv = stats.ttest_ind(pic_data[pic_data["T-P"] == "VH"]["PV_2"], pic_data[pic_data["T-P"] == "H"]["PV_2"])
print(str(pv/2))
print(pic_data[pic_data["T-P"] == "B"]["PV_2"].mean())
print(pic_data[pic_data["T-P"] == "H"]["PV_2"].mean())
print(pic_data[pic_data["T-P"] == "VH"]["PV_2"].mean())

1.5243026056067067e-08
4.789029535864979
5.008658008658009
5.755364806866953


#### Text

In [19]:
tv,pv = stats.ttest_ind(txt_data[txt_data["T-T"] == "H"]["TV_2"], txt_data[txt_data["T-T"] == "VH"]["TV_2"])
print(str(pv/2))
print(pic_data[pic_data["T-T"] == "L"]["TV_2"].mean())
print(pic_data[pic_data["T-T"] == "B"]["TV_2"].mean())
print(pic_data[pic_data["T-T"] == "H"]["TV_2"].mean())
print(pic_data[pic_data["T-T"] == "VH"]["TV_2"].mean())

0.0009379015136865651
2.5172413793103448
3.6357615894039736
5.421052631578948
5.940397350993377


### Review 3
#### Picture

In [20]:
tv,pv = stats.ttest_ind(pic_data[pic_data["T-P"] == "VH"]["PV_3"], pic_data[pic_data["T-P"] == "H"]["PV_3"])
print(str(pv/2))
print(pic_data[pic_data["T-P"] == "B"]["PV_3"].mean())
print(pic_data[pic_data["T-P"] == "H"]["PV_3"].mean())
print(pic_data[pic_data["T-P"] == "VH"]["PV_3"].mean())

0.003932695306409912
4.476793248945148
5.0606060606060606
5.459227467811159


#### Text

In [21]:
tv,pv = stats.ttest_ind(txt_data[txt_data["T-T"] == "H"]["TV_3"], txt_data[txt_data["T-T"] == "VH"]["TV_3"])
print(str(pv/2))
print(pic_data[pic_data["T-T"] == "L"]["TV_3"].mean())
print(pic_data[pic_data["T-T"] == "B"]["TV_3"].mean())
print(pic_data[pic_data["T-T"] == "H"]["TV_3"].mean())
print(pic_data[pic_data["T-T"] == "VH"]["TV_3"].mean())

0.00011018744094783085
2.43448275862069
3.410596026490066
5.2330827067669174
5.728476821192053


## PRH & CPF
> `CPFM` includes inverse items

In [22]:
data["PRH_1"] = (data["PRH1_1"]+data["PRH2_1"]+data["PRH3_1"])/3
data["PRH_2"] = (data["PRH1_2"]+data["PRH2_2"]+data["PRH3_2"])/3
data["PRH_3"] = (data["PRH1_3"]+data["PRH2_3"]+data["PRH3_3"])/3
data["CPF_1"] = (data["CPF1_1"]+data["CPF2_1"]+data["CPF3_1"])/3
data["CPF_2"] = (data["CPF1_2"]+data["CPF2_2"]+data["CPF3_2"])/3
data["CPF_3"] = (data["CPF1_3"]+data["CPF2_3"]+data["CPF3_3"])/3
# data["CPFM_1"] = (data["CPF1_1"]+data["CPF2_1"]+data["CPF3_1"]+data["CPF4_1"]+data["CPF5_1"])/5
# data["CPFM_2"] = (data["CPF1_2"]+data["CPF2_2"]+data["CPF3_2"]+data["CPF4_2"]+data["CPF5_2"])/5
# data["CPFM_3"] = (data["CPF1_3"]+data["CPF2_3"]+data["CPF3_3"]+data["CPF4_3"]+data["CPF5_3"])/5

data["PRH"] = (data["PRH_1"] + data["PRH_2"] + data["PRH_3"]) / 3
data["PRH1"] = (data["PRH_1"] + data["PRH_2"]) / 2
data["CPF"] = (data["CPF_1"] + data["CPF_2"] + data["CPF_3"]) / 3
# data["CPFM"] = (data["CPFM_1"] + data["CPFM_2"] + data["CPFM_3"]) / 3

### Review 1

In [23]:
# print("Mean of PRH = %.2f" % data[data["Group"] == 1]["PRH_1"].mean()) 
# print("Mean of CPF = %.2f" % data[data["Group"] == 1]["CPF_1"].mean())
# print("Mean of CPFM = %.2f" % data[data["Group"] == 1]["CPFM_1"].mean())
# print(stats.norm.interval(0.90, loc = data[data["Group"] == 1]["PRH_1"].mean(), scale = data[data["Group"] == 1]["PRH_1"].std()))
# print(stats.norm.interval(0.95, loc = data[data["Group"] == 1]["PRH_1"].mean(), scale = data[data["Group"] == 1]["PRH_1"].std()))
# print(stats.t.interval(0.90, df = len(data[data["Group"] == 1]["PRH_1"]),loc = data[data["Group"] == 1]["PRH_1"].mean(), scale = data[data["Group"] == 1]["PRH_1"].std()))\
# data[data["Group"] == i]["PRH_1"].std()
# len(data[data["Group"] == i]["PRH_1"]) - 1
# math.sqrt(len(data[data["Group"] == i]["PRH_1"]) - 1)

# data[data["Group"] == i]["PRH_1"].std() / math.sqrt(len(data[data["Group"] == i]["PRH_1"]) - 1)



In [24]:
for i in range(1,20):
    # print("\nGroup %i" % i)
    # print(str(data[data["Group"] == i]["PRH_1"].mean()) + " (" + str(data[data["Group"] == i]["PRH_1"].std()) + ")") 
    print("%.3f (%.3f)" % (data[data["Group"] == i]["PRH_2"].mean(), data[data["Group"] == i]["PRH_2"].std() / math.sqrt(2)))
    # print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["PRH_1"].mean(), scale = data[data["Group"] == i]["PRH_1"].std() / math.sqrt(len(data[data["Group"] == i]["PRH_1"]) - 1)))
    # print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["CPF_1"].mean(), scale = data[data["Group"] == i]["CPF_1"].std() / math.sqrt(len(data[data["Group"] == i]["CPF_1"]) - 1)))
    # print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["CPFM_1"]) - 1, loc = data[data["Group"] == i]["CPFM_1"].mean(), scale = data[data["Group"] == i]["CPFM_1"].std() / math.sqrt(len(data[data["Group"] == i]["CPFM_1"]) - 1)) )
    # print("\n")

    # print("Mean of CPF = %.2f" % data[data["Group"] == i]["CPF_1"].mean())
    # print("Std of PRH = %.2f" % data[data["Group"] == i]["PRH_1"].std())
    # print("%.3f (%.3f)" % (data[data["Group"] == i]["CPF_1"].mean(), data[data["Group"] == i]["CPF_1"].std()))
    # print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["CPF_1"].mean(), scale = data[data["Group"] == i]["CPF_1"].std() / math.sqrt(len(data[data["Group"] == i]["CPF_1"]) - 1)))
    # print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["CPF_1"].mean(), scale = data[data["Group"] == i]["CPF_1"].std() / math.sqrt(len(data[data["Group"] == i]["CPF_1"]) - 1)))
    # print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["CPF_1"]) - 1, loc = data[data["Group"] == i]["CPF_1"].mean(), scale = data[data["Group"] == i]["CPF_1"].std() / math.sqrt(len(data[data["Group"] == i]["CPF_1"]) - 1)))
    # print("\n")

    # print("Mean of CPFM = %.2f" % data[data["Group"] == i]["CPFM_1"].mean())
    # print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["CPFM_1"].mean(), scale = data[data["Group"] == i]["CPFM_1"].std() / math.sqrt(len(data[data["Group"] == i]["CPFM_1"]) - 1)))
    # print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["CPFM_1"].mean(), scale = data[data["Group"] == i]["CPFM_1"].std() / math.sqrt(len(data[data["Group"] == i]["CPFM_1"]) - 1)))
    # print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["CPFM_1"]) - 1, loc = data[data["Group"] == i]["CPFM_1"].mean(), scale = data[data["Group"] == i]["CPFM_1"].std() / math.sqrt(len(data[data["Group"] == i]["CPFM_1"]) - 1)))

4.514 (1.152)
4.796 (1.135)
4.793 (1.010)
4.360 (1.062)
5.240 (0.888)
4.837 (0.972)
5.056 (0.954)
4.824 (1.008)
5.341 (0.708)
5.403 (0.776)
5.485 (0.830)
5.268 (0.984)
5.444 (0.938)
5.321 (0.854)
5.607 (0.766)
5.260 (0.991)
3.367 (1.117)
4.220 (1.002)
4.717 (0.881)


### Review 2

In [ ]:
for i  in range(1,20):
    print("\nGroup %i" % i)
    print("Mean of PRH = %.2f" % data[data["Group"] == i]["PRH_2"].mean()) 
    print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["PRH_2"].mean(), scale = data[data["Group"] == i]["PRH_2"].std()))
    print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["PRH_2"].mean(), scale = data[data["Group"] == i]["PRH_2"].std()))
    print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["PRH_2"]) - 1, loc = data[data["Group"] == i]["PRH_2"].mean(), scale = data[data["Group"] == i]["PRH_2"].std()))
    print("\n")

    print("Mean of CPF = %.2f" % data[data["Group"] == i]["CPF_1"].mean())
    print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["CPF_2"].mean(), scale = data[data["Group"] == i]["CPF_2"].std()))
    print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["CPF_2"].mean(), scale = data[data["Group"] == i]["CPF_2"].std()))
    print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["CPF_2"]) - 1, loc = data[data["Group"] == i]["CPF_2"].mean(), scale = data[data["Group"] == i]["CPF_2"].std()))
    print("\n")

    print("Mean of CPFM = %.2f" % data[data["Group"] == i]["CPFM_2"].mean())
    print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["CPFM_2"].mean(), scale = data[data["Group"] == i]["CPFM_2"].std()))
    print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["CPFM_2"].mean(), scale = data[data["Group"] == i]["CPFM_2"].std()))
    print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["CPFM_2"]) - 1, loc = data[data["Group"] == i]["CPFM_2"].mean(), scale = data[data["Group"] == i]["CPFM_2"].std()))

### Review 3

In [ ]:
for i  in range(1,20):
    print("\nGroup %i" % i)
    print("Mean of PRH = %.2f" % data[data["Group"] == i]["PRH_3"].mean()) 
    print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["PRH_3"].mean(), scale = data[data["Group"] == i]["PRH_3"].std()))
    print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["PRH_3"].mean(), scale = data[data["Group"] == i]["PRH_3"].std()))
    print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["PRH_3"]) - 1, loc = data[data["Group"] == i]["PRH_3"].mean(), scale = data[data["Group"] == i]["PRH_3"].std()))
    print("\n")

    print("Mean of CPF = %.2f" % data[data["Group"] == i]["CPF_3"].mean())
    print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["CPF_3"].mean(), scale = data[data["Group"] == i]["CPF_3"].std()))
    print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["CPF_3"].mean(), scale = data[data["Group"] == i]["CPF_3"].std()))
    print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["CPF_3"]) - 1, loc = data[data["Group"] == i]["CPF_3"].mean(), scale = data[data["Group"] == i]["CPF_3"].std()))
    print("\n")

    print("Mean of CPFM = %.2f" % data[data["Group"] == i]["CPFM_3"].mean())
    print(stats.norm.interval(0.90, loc = data[data["Group"] == i]["CPFM_3"].mean(), scale = data[data["Group"] == i]["CPFM_3"].std()))
    print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["CPFM_3"].mean(), scale = data[data["Group"] == i]["CPFM_3"].std()))
    print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["CPFM_3"]) - 1, loc = data[data["Group"] == i]["CPFM_3"].mean(), scale = data[data["Group"] == i]["CPFM_3"].std()))


Group 1
Mean of PRH = 5.24
(2.7682777477016636, 7.707912728488812)
(2.295126179790501, 8.181064296399976)
(2.7206829598615845, 7.7555075163288905)


Mean of CPF = 5.25
(2.831756171851468, 7.671645188692748)
(2.368158945388078, 8.135242415156139)
(2.785122464741114, 7.718278895803103)


Mean of CPFM = 4.84
(2.5745570080933815, 7.107075644967839)
(2.1404017905590753, 7.541230862502147)
(2.5308849020081525, 7.150747751053068)

Group 2
Mean of PRH = 5.14
(3.1598502376755224, 7.114659566246046)
(2.781031915698086, 7.493477888223484)
(3.1232664110939656, 7.151243392827603)


Mean of CPF = 5.12
(3.3163891233660507, 6.91890499428101)
(2.9713158458105022, 7.263978271836559)
(3.2830641752005985, 6.952229942446462)


Mean of CPFM = 4.69
(2.887049452627798, 6.501185841489844)
(2.5408630834555637, 6.847372210662078)
(2.8536170092639574, 6.534618284853684)

Group 3
Mean of PRH = 5.17
(3.2200431733100237, 7.113290160023309)
(2.847121707818406, 7.486211625514928)
(3.1884457332793055, 7.14488760005402

In [ ]:
for i in range(1,20):
    print("%.3f (%.3f)" % (data[data["Group"] == i]["CPFM"].mean(), data[data["Group"] == i]["CPFM"].std()))
print("\n")
for i in range(1,20):
    print(stats.t.interval(0.90, df = len(data[data["Group"] == i]["CPFM"]) - 1, loc = data[data["Group"] == i]["CPFM"].mean(), 
    scale = data[data["Group"] == i]["CPFM"].std() / math.sqrt(len(data[data["Group"] == i]["CPFM"]) - 1)))
print("\n")
for i in range(1,20):
    print(stats.norm.interval(0.95, loc = data[data["Group"] == i]["CPFM"].mean(), 
    scale = data[data["Group"] == i]["CPFM"].std() / math.sqrt(len(data[data["Group"] == i]["CPFM"]) - 1)))

4.093 (1.070)
4.258 (0.960)
4.129 (0.991)
4.172 (1.118)
4.696 (0.823)
4.516 (0.863)
4.632 (0.886)
4.547 (1.001)
5.076 (0.782)
4.940 (0.889)
5.000 (0.848)
4.836 (0.827)
5.257 (0.800)
5.118 (0.813)
5.014 (0.948)
5.104 (0.997)
3.374 (0.958)
3.536 (1.218)
4.148 (1.220)


(3.8333932498777914, 4.351640763727649)
(4.030009444809192, 4.48502323492937)
(3.9091539990931348, 4.34831726527468)
(3.912785230878549, 4.431868857171764)
(4.503082977509033, 4.88922471479866)
(4.288932330810491, 4.742813700935539)
(4.439409051143287, 4.825035393301159)
(4.3271753893914315, 4.766609921343031)
(4.868309822950762, 5.284535705504524)
(4.725034395038288, 5.155237713805245)
(4.782533044704172, 5.217466955295826)
(4.655736603541685, 5.0153745075694225)
(5.084210137693963, 5.429451064491832)
(4.93455760798598, 5.301156677728304)
(4.767855014352848, 5.260051962391336)
(4.872850543250577, 5.335954488195961)
(3.118927458586511, 3.6290400210882856)
(3.224216017321825, 3.8485112554054473)
(3.823112894841954, 4.472822

In [ ]:
statistic, p_value = f_oneway(data[data["Group"] == 1]["PRH1"], 
data[data["Group"] == 2]["PRH1"], data[data["Group"] == 3]["PRH1"])
print("P-value:", p_value)

P-value: 0.2601074443720693


In [ ]:
model = smf.ols(formula="PRH_3 ~ PrePV_3 + PreTV_3 + Gender + Age + Education + Income + Race + Ideology + Satiety", data=data).fit(cov_type='HC3')
print(model.summary())
# 'Gender','Age', 'Education', 'Income', 'Race', 'Ideology', 'Satiety'

                            OLS Regression Results                            
Dep. Variable:                  PRH_3   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.023
Date:                Sat, 20 Jan 2024   Prob (F-statistic):              0.435
Time:                        10:13:19   Log-Likelihood:                -927.96
No. Observations:                 580   AIC:                             1932.
Df Residuals:                     542   BIC:                             2098.
Df Model:                          37                                         
Covariance Type:                  HC3                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [ ]:
data_pictxt = pic_data[pic_data["T-T"] != "N"]
data_pictxt["Match_1"] = abs(data_pictxt["PrePV_1"] - data_pictxt["PreTV_1"])
data_pictxt["Match_2"] = abs(data_pictxt["PrePV_2"] - data_pictxt["PreTV_2"])
data_pictxt["Match_3"] = abs(data_pictxt["PrePV_3"] - data_pictxt["PreTV_3"])
data_pictxt["PRH_1"] = (data_pictxt["PRH1_1"]+data_pictxt["PRH2_1"]+data_pictxt["PRH3_1"])/3
data_pictxt["PRH_2"] = (data_pictxt["PRH1_2"]+data_pictxt["PRH2_2"]+data_pictxt["PRH3_2"])/3
data_pictxt["PRH_3"] = (data_pictxt["PRH1_3"]+data_pictxt["PRH2_3"]+data_pictxt["PRH3_3"])/3
data_pictxt["PRH_12"] = (data_pictxt["PRH_1"] + data_pictxt["PRH_2"]) / 2

In [ ]:
data_pictxt["Match_1"] = abs(data_pictxt["PrePV_1"] - data_pictxt["PreTV_1"])
data_pictxt["Match_2"] = abs(data_pictxt["PrePV_2"] - data_pictxt["PreTV_2"])
data_pictxt["Match_3"] = abs(data_pictxt["PrePV_3"] - data_pictxt["PreTV_3"])


In [ ]:
data_pictxt["Match_1"] = abs(data_pictxt["PrePV_1"] - data_pictxt["PreTV_1"]) / data_pictxt["PrePV_1"]
data_pictxt["Match_2"] = abs(data_pictxt["PrePV_2"] - data_pictxt["PreTV_2"]) / data_pictxt["PrePV_2"]
data_pictxt["Match_3"] = abs(data_pictxt["PrePV_3"] - data_pictxt["PreTV_3"]) / data_pictxt["PrePV_3"]

In [ ]:
data_pictxt["PRH"] = data_pictxt["PRH_1"] + data_pictxt["PRH_2"]
data_pictxt['Source'] = 'R1'
data_pictxt.loc[data_pictxt.index[data_pictxt['PRH_1'].count():], 'Source'] = 'R2'
data_pictxt['Source']

In [ ]:
model = smf.ols(formula = "PRH_2 ~ Match_2 + PrePV_2 + PreTV_2 + Gender + Age + Education + Income + Race + Ideology + Satiety", data = data_pictxt).fit(cov_type='HC3')

# model1 = smf.ols(formula="PRH_2 ~ Match_2 + PrePV_2 + PreTV_2 + Gender + Age + Education + Income + Race + Ideology + Satiety",
#                data=data_pictxt).fit(cov_type='cluster', cov_kwds={'groups': data_pictxt['Group']})
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  PRH_2   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     1.740
Date:                Sat, 20 Jan 2024   Prob (F-statistic):            0.00472
Time:                        10:48:59   Log-Likelihood:                -954.89
No. Observations:                 580   AIC:                             1988.
Df Residuals:                     541   BIC:                             2158.
Df Model:                          38                                         
Covariance Type:                  HC3                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [ ]:
model = smf.ols(formula="PRH_3 ~ Match_3 + Gender + Age + Education + Income + Race + Ideology + Satiety", data=data).fit(cov_type='HC3')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  PRH_2   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.394
Date:                Fri, 19 Jan 2024   Prob (F-statistic):             0.0658
Time:                        21:06:48   Log-Likelihood:                -1592.3
No. Observations:                 908   AIC:                             3257.
Df Residuals:                     872   BIC:                             3430.
Df Model:                          35                                         
Covariance Type:                  HC3                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [4]:
data_combined = pd.read_csv("New.csv")
data_combined.columns
data_combined["Satiety"].describe()

count    2724.000000
mean        3.963656
std         1.277708
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         7.000000
Name: Satiety, dtype: float64

In [5]:
data_pictxt_c = data_combined.dropna()
print(data_pictxt_c.columns)

Index(['R', 'Group', 'T-P', 'T-T', 'Pid', 'PRH1', 'PRH2', 'PRH3', 'CPFM1',
       'CPF1', 'CPF2', 'CPFM2', 'CPF3', 'PRC1', 'PRC2', 'PRC3', 'PRC4',
       'PRCM1', 'PRE1', 'PRE2', 'PR1', 'PR2', 'PV', 'TV', 'PrePV', 'PreTV',
       'Gender', 'Age', 'Education', 'Income', 'Race', 'Ideology', 'Satiety'],
      dtype='object')


In [ ]:
data_pictxt_c["PRH"] = (data_pictxt_c["PRH1"]+data_pictxt_c["PRH2"]+data_pictxt_c["PRH3"])/3
data_pictxt_c["Match"] = abs(data_pictxt_c["PrePV"] - data_pictxt_c["PreTV"])
data_pictxt_c["Match1"] = abs(data_pictxt_c["PrePV"] - data_pictxt_c["PreTV"]) / data_pictxt_c["PreTV"]
data_pictxt_c["Match2"] = abs(data_pictxt_c["PrePV"] - data_pictxt_c["PreTV"]) / data_pictxt_c["PrePV"]
data_pictxt_c["Match3"] = abs(np.sqrt(data_pictxt_c["PrePV"]) - np.sqrt(data_pictxt_c["PreTV"]))
data_pictxt_c["Match4"] = 2 * abs(data_pictxt_c["Match"]) / (data_pictxt_c["PrePV"] + data_pictxt_c["PreTV"])
data_pictxt_c["PxT"] = data_pictxt_c["PrePV"] * data_pictxt_c["PreTV"]
data_pictxt_c["SV"] = 2 * abs(4 - data_pictxt_c["Satiety"]) + 1
data_pictxt_c["SMP1"] = abs(data_pictxt_c["PrePV"] - data_pictxt_c["SV"])
data_pictxt_c["SMP2"] = abs(data_pictxt_c["PrePV"] - data_pictxt_c["Satiety"])
data_pictxt_c["SMT1"] = abs(data_pictxt_c["PreTV"] - data_pictxt_c["SV"])
data_pictxt_c["SMT2"] = abs(data_pictxt_c["PreTV"] - data_pictxt_c["Satiety"])
data_pictxt_c["SxT"] = data_pictxt_c["SV"] * data_pictxt_c["PreTV"]
data_pictxt_c["SxP"] = data_pictxt_c["SV"] * data_pictxt_c["PrePV"]
data_pictxt_c["CPF"] = (data_pictxt_c["CPF1"] + data_pictxt_c["CPF2"] + data_pictxt_c["CPF3"])/3
data_pictxt_c["CPFM"] = (data_pictxt_c["CPF1"] + data_pictxt_c["CPF2"] + data_pictxt_c["CPF3"] + 7 - data_pictxt_c["CPFM1"] + 7 - data_pictxt_c["CPFM2"])/5
tp = data_pictxt_c

In [ ]:
data_pictxt_c2 = data_pictxt_c[data_pictxt_c["R"] != "R3"]
data_pictxt_c2["PRH"] = (data_pictxt_c2["PRH1"]+data_pictxt_c2["PRH2"])/2
tp12 = data_pictxt_c2

In [14]:
d = data_pictxt_c[data_pictxt_c["T-P"] != "H"]
d123 = d[d["T-T"] != "H"]

d0 = data_pictxt_c2[data_pictxt_c2["T-P"] != "H"]
d12 = d0[d0["T-T"] != "H"]

r1 = d12[d12["R"] == "R1"]
r2 = d12[d12["R"] == "R2"]

# Datasets: tp, tp12, d123, d12, r1, r2
# Variables: Match - Match3, PxT, SV, SMP1-2, SMT1-2
# cov_type='cluster', cov_kwds={'groups': dataset['Pid']}

In [19]:
p1 = Process(data=tp, model=4,x=["Match2"],y="PRH",m=["CPF"],
            controls=["Satiety","PR1","PRE1","PreTV","PrePV"],controls_in="all_to_y")
p1.summary()

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Match2
    y = PRH
    m1 = CPF
Statistical Controls:
 Satiety, PR1, PRE1, PreTV, PrePV



Sample size:
1740

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = PRH 
OLS Regression Summary

     R²  Adj. R²    MSE        F  df1   df2  p-value
 0.4316   0.4289 1.1106 187.8435    7  1732   0.0000

Coefficients

         coeff     se       t      p    LLCI   ULCI
Cons    0.4594 0.3205  1.4335 0.1519 -0.1687 1.0875
Match2  0.1845 0.1949  0.9469 0.3438 -0.1974 0.5664
Satiety 0.0312 0.0195  1.5958 0.1107 -0.0071 0.0695
PR1     0.1491 0.0218  6.8520 0.0000  0.1064 0.1917
PRE1    0.2377 0.0209

c:\Users\wxy\anaconda3\envs\myenv\lib\pyprocessmacro\utils.py:33: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  llci = np.percentile(samples, plow * 100, interpolation="lower")
c:\Users\wxy\anaconda3\envs\myenv\lib\pyprocessmacro\utils.py:34: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  ulci = np.percentile(samples, phigh * 100, interpolation="higher")


In [76]:
model_c = smf.ols(formula="PRH ~ Match4 + PrePV + PreTV + PxT + Gender + Age + Education + Income + Race + Ideology + Satiety",
    data = tp).fit(cov_type='cluster', cov_kwds={'groups': tp['Pid']})
print(model_c.summary())

                            OLS Regression Results                            
Dep. Variable:                    PRH   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                 1.533e+11
Date:                Mon, 22 Jan 2024   Prob (F-statistic):               0.00
Time:                        11:11:49   Log-Likelihood:                -2948.2
No. Observations:                1740   AIC:                             5966.
Df Residuals:                    1705   BIC:                             6158.
Df Model:                          34                                         
Covariance Type:              cluster                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

c:\Users\wxy\anaconda3\envs\myenv\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 34, but rank is 33
  warnings.warn('covariance of constraints does not have full '


In [ ]:
# Edit here
model_c = smf.mixedlm(formula="PRH ~ Match4 + PrePV + PreTV + PxT + Gender + Age + Education + Income + Race + Ideology + Satiety",
    data = tp, groups = tp['Pid'], cov_type='cluster', cov_kwds={'groups': tp['Pid']}).fit()
print(model_c.summary())

In [ ]:
model_r = smf.ols(formula="PRH ~ Match1 + SV + PrePV + PreTV + SMP2 + SMT2 + Age + Income + Satiety",
                data = r2).fit(cov_type='cluster', cov_kwds={'groups': r2['Pid']})
print(model_r.summary())

                            OLS Regression Results                            
Dep. Variable:                    PRH   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     4.007
Date:                Mon, 22 Jan 2024   Prob (F-statistic):           4.53e-06
Time:                        00:05:36   Log-Likelihood:                -534.86
No. Observations:                 305   AIC:                             1098.
Df Residuals:                     291   BIC:                             1150.
Df Model:                          13                                         
Covariance Type:              cluster                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [ ]:
model_r = smf.ols(formula="PRH ~ PrePV + PreTV + SMP1 + SMT1 + Gender + Age + Education + Income + Ideology + Race",
                data=r2).fit(cov_type='HC1')
print(model_r.summary())

                            OLS Regression Results                            
Dep. Variable:                    PRH   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     38.91
Date:                Mon, 22 Jan 2024   Prob (F-statistic):           4.93e-80
Time:                        00:27:07   Log-Likelihood:                -521.23
No. Observations:                 305   AIC:                             1108.
Df Residuals:                     272   BIC:                             1231.
Df Model:                          32                                         
Covariance Type:                  HC1                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

c:\Users\wxy\anaconda3\envs\myenv\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 33, but rank is 29
  warnings.warn('covariance of constraints does not have full '


In [ ]:
A = data_pictxt_c2[data_pictxt_c2["R"] == "R1"]["Match"]
A_1 = data_pictxt_c2[data_pictxt_c2["R"] == "R1"]["PreTV"]
A_2 = data_pictxt_c2[data_pictxt_c2["R"] == "R1"]["PrePV"]
B = data_pictxt_c2[data_pictxt_c2["R"] == "R2"]["Match"]
B_1 = data_pictxt_c2[data_pictxt_c2["R"] == "R2"]["PreTV"]
B_2 = data_pictxt_c2[data_pictxt_c2["R"] == "R2"]["PrePV"]
print(A.describe(), "\n")
print(A_1.describe(), "\n")
print(A_2.describe(), "\n")

count    580.000000
mean       1.249828
std        0.795697
min        0.190000
25%        0.720000
50%        1.330000
75%        1.880000
max        2.810000
Name: Match, dtype: float64 

count    580.000000
mean       4.799414
std        1.388580
min        2.600000
25%        4.167500
50%        4.690000
75%        6.210000
max        6.210000
Name: PreTV, dtype: float64 

count    580.000000
mean       4.869552
std        0.443224
min        4.330000
25%        4.330000
50%        4.880000
75%        5.410000
max        5.410000
Name: PrePV, dtype: float64 



In [ ]:
print(B.describe(), "\n")
print(B_1.describe(), "\n")
print(B_2.describe(), "\n")

count    580.000000
mean       1.815293
std        1.297673
min        0.180000
25%        0.880000
50%        1.530000
75%        2.692500
max        4.350000
Name: Match, dtype: float64 

count    580.000000
mean       4.264621
std        1.840068
min        1.680000
25%        3.000000
50%        3.440000
75%        6.210000
max        6.210000
Name: PreTV, dtype: float64 

count    580.000000
mean       5.342155
std        0.554019
min        4.680000
25%        4.680000
50%        5.330000
75%        6.030000
max        6.030000
Name: PrePV, dtype: float64 

